In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('tsdm.csv')
df.info()

## 3.1 Preprocessing

In [ ]:
# convert date type of OBSERVATION_DATE (datetime)
df['OBSERVATION_DATE'] = pd.to_datetime(df['OBSERVATION_DATE'])

# sort: essential time series prediction
df = df.sort_values(['PADDOCK_ID','OBSERVATION_DATE']).reset_index(drop=True)

df.info()

In [ ]:
df.groupby("PADDOCK_ID").size()

### Function

- create_sequence(sequence, lookback, forecast_horizon, target_col)
- data_prep(df, feature_columns, lookback, test_steps, target_col)
- MyLSTMNet(nn.Module)
- train_predict_model(model, n_epochs, lr, X_all, y_all, lengths, validation_split=0.2)
- pred_eval(model, X, y, lengths, train_d, test_d, lookback, target_col)

In [ ]:
# create_sequences function
def create_sequences(sequence, lookback, forecast_horizon, target_col):
    T, num_features = sequence.shape
    X, y, lengths = [], [], []
    pad_vector = np.zeros((lookback, num_features))

    # Fixed-length lookback with pre-padding
    for t in range(1, T - forecast_horizon + 1):
        context = sequence[:t]
        if len(context) > lookback:
            context = context[-lookback:]

        padded_context = pad_vector.copy()
        padded_context[-len(context):] = context

        X.append(padded_context)
        y.append(sequence[t:t + forecast_horizon, target_col])
        lengths.append(min(len(context), lookback))

    return np.array(X), np.array(y), lengths

In [ ]:
# data_prep function: each location, split the data, scaler
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

def data_prep(df, feature_columns, lookback, test_steps, target_col):
    # prepare to store all training data
    X_all, y_all = [], []
    location_ids = [] # to track which location each sample comes from
    test_data = [] # to store test data for each location
    train_data = []
    lengths_all = []

    # Fit a global scaler
    all_train_values = []
    for _, group in df.groupby("PADDOCK_ID"):
        feature_values = group[feature_columns].values

        if len(feature_values) > lookback + test_steps:
            all_train_values.append(feature_values[:-test_steps])
        all_train_values = np.vstack(all_train_values)

        global_scaler = MinMaxScaler()
        global_scaler.fit(all_train_values)

        for location_id, group in df.groupby("PADDOCK_ID"):
            feature_values = group[feature_columns].values

            if len(feature_values) <= 194:
                continue

            # split and scale
            train_sample = global_scaler.transform(feature_values[:-test_steps])
            test_sample = global_scaler.transform(feature_values[-test_steps:])

            train_data.append((location_id, train_sample))
            test_data.append((location_id, test_sample, global_scaler))

            # prepare LSTM sequence data for training
            X_location, y_location, lengths = create_sequences(train_sample, lookback, test_steps, target_col)

            # append to the overall dataset
            X_all.append(X_location)
            y_all.append(y_location)
            lengths_all.append(lengths)

            # store location ID for tracking
            location_ids.extend([location_id] * len(y_location))

        # concatenate all locations' training data for model training
        X_all = np.concatenate(X_all, axis=0)
        y_all = np.concatenate(y_all, axis=0)
        lengths_all = np.concatenate(lengths_all, axis=0)

        X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], X_all.shape[2]))

        return(torch.Tensor(X_all), torch.Tensor(y_all),
               torch.Tensor(lengths_all), train_data, test_data)
        

In [ ]:
# Defining the LSTM network

import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MyLSTMNet(nn.Module):
    def __init__(self, num_features, hidden_layer_size, num_layers, output_size, dropout_prob):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_layer_size,
            num_layers=num_layers,
            batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, data, lengths):
        packed_data = pack_padded_sequence(data, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # Run through LSTM
        packed_output, (hn, cn) = self.lstm(packed_data)

        # Use the last layer's hidden state
        last_hidden = hn[-1]

        # apply dropout and final linear layer
        out = self.dropout(last_hidden)
        out = self.fc(out)
        return out

In [ ]:
# Defining Training Process
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, random_split

def train_predict_model(model, n_epochs, lr, X_all, y_all, lengths, validation_split=0.2):
    batch_size = 32

    # split data into train and validation sets
    dataset = TensorDataset(X_all, y_all, lengths)
    val_size = int(len(dataset) * validation_split)
    train_size = len(dataset) - val_size
    train_set, val_set = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"The model has {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters")

    train_loss_history = []
    val_loss_history = []

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(n_epochs):
        model.train()
        for X_batch, y_batch, lengths_batch in train_loader:
            y_pred = model(X_batch, lengths_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # validation check every 100 epochs
        if epoch % 100 == 0:
            model.eval()
            with torch.no_grad():
                train_preds = model(X_all[train_set.indices], lengths[train_set.indices])
                train_loss = loss_fn(train_preds, y_all[train_set.indices]).item()

                val_preds = model(X_all[val_set.indices], lengths[val_set.indices])
                val_loss = loss_fn(val_preds, y_all[val_set.indices]).item()

                print(f"Epoch {epoch+1}: train loss {train_loss:.4f}, val_loss {val_loss:.4f}")

                train_loss_history.append(train_loss)
                val_loss_history.append(val_loss)

                # save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model_state = model.state_dict()

    # restore best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return train_loss_history, val_loss_history, model
    

In [ ]:
# visualisation of train loss
def vis_train_loss(train_loss_history, val_loss_history):
    epochs = range(0, n_epochs, 100)
    plt.plot(epochs, train_loss_history, label='Training Loss')
    plt.plot(epochs, val_loss_history, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Convergence')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# computing the RMSE: root_mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

def pred_eval(model, X, y, lengths, train_d, test_d, lookback, target_col):
    model.eval()
    with torch.no_grad():
        train_preds = model(X, lengths)
        print("Training RMSE:", root_mean_squared_error(y.flatten().tolist(), train_preds.flatten().tolist()))
        print("Training R2:", r2_score(y.flatten().tolist(), train_preds.flatten().tolist()))

        X_test = []
        y_test = []
        lengths_test = []

        for count, (location_id, test_values, scaler) in enumerate(test_d):
            train_values = train_d[count][1]
            X_test.append(train_values[-lookback:])
            y_test.append(test_values[:, target_col])

            # append the actual lengths (just like the training phase)
            lengths_test.append(len(train_values[-lookback:]))

        X_test = torch.Tensor(np.array(X_test))
        y_test = torch.Tensor(np.array(y_test))
        lengths_test = torch.Tensor(lengths_test).long()
        test_preds = model(X_test, lengths_test)

        print("Test RMSE:", root_mean_squared_error(y_test.flatten().tolist(), test_preds.flatten().tolist()))
        print("Test R2:", r2_score(y_test.flatten().tolist(), test_preds.flatten().tolist()))
        
        plt.figure(figsize = (10, 6))
        plt.plot(y_test.flatten().tolist(), label="Expected Value")
        plt.plot(test_preds.flatten().tolist(), label="Predicted Value")
        plt.grid()
        plt.legend(fontsize=10)
        plt.tight_layout()
        plt.show()

In [ ]:
# ensure the last 5 timesteps of each paddock for test: test_steps=5

lookback = 5
test_steps = 5
target_col = 0
X_5, y_5, lengths_5, train_d_5, test_d_5 = data_prep(df, ['TSDM'], lookback, test_steps, target_col)

print("Shape of input data after sequence creation:", X_5.shape)
print("Shape of targets after sequence creation:", y_5.shape)

## 3.2 Univariate LSTM Model 1

In [ ]:
# Univariate LSTM model (lookback=5, predict=5)
num_features = X_5.shape[2]
hidden_layer_size = 10
output_size = test_steps
num_layers = 2
dropout_prob = 0.2
model_lstm_5 = MyLSTMNet(num_features, hidden_layer_size, num_layers, output_size, dropout_prob)

print(model_lstm_5)
print("============================================================")

# training the Univariate LSTM Model
n_epochs = 201
lr = 0.001
train_loss_history_5, val_loss_history_5, model_lstm_5 = train_predict_model(model_lstm_5, n_epochs, lr, X_5, y_5, lengths_5)

# visualisation of train loss
vis_train_loss(train_loss_history_5, val_loss_history_5)
print( )

# RMSE of Univariate LSTM Model(lookback=5, predict=5)
lookback = 5
target_col = 0
pred_eval(model_lstm_5, X_5, y_5, lengths_5, train_d_5, test_d_5, lookback, target_col)


In [ ]:
# try to find optimal hyperparameters

lookback = 5
test_steps = 5  # ensure the last 5 timesteps of each paddock for test: test_steps=5
target_col = 0
X_5, y_5, lengths_5, train_d_5, test_d_5 = data_prep(df, ['TSDM'], lookback, test_steps, target_col)

print("Shape of input data after sequence creation:", X_5.shape)
print("Shape of targets after sequence creation:", y_5.shape)
print("============================================================")

# Univariate LSTM model (lookback=5, predict=5)
num_features = X_5.shape[2]
hidden_layer_size = 15
output_size = test_steps
num_layers = 1 # to check more simply and reduce overfiting risk
dropout_prob = 0.2
model_lstm_5 = MyLSTMNet(num_features, hidden_layer_size, num_layers, output_size, dropout_prob)

print(model_lstm_5)
print("============================================================")

# training the Univariate LSTM Model
n_epochs = 201
lr = 0.001
train_loss_history_5, val_loss_history_5, model_lstm_5 = train_predict_model(model_lstm_5, n_epochs, lr, X_5, y_5, lengths_5)

# visualisation of train loss
vis_train_loss(train_loss_history_5, val_loss_history_5)
print( )

# RMSE of Univariate LSTM Model(lookback=5, predict=5)
lookback = 5
target_col = 0
pred_eval(model_lstm_5, X_5, y_5, lengths_5, train_d_5, test_d_5, lookback, target_col)


## 3.3 Univariate LSTM Model 2

In [ ]:
# Univariate LSTM Model (Lookback=10, Predict=5)
lookback = 10
test_steps = 5
target_col = 0
X_10, y_10, lengths_10, train_d_10, test_d_10 = data_prep(df, ['TSDM'], lookback, test_steps, target_col)

print("Shape of input data after sequence creation:", X_10.shape)
print("Shape of targets after sequence creation:", y_10.shape)
print("============================================================")

num_features = X_10.shape[2]
hidden_layer_size = 15
output_size = test_steps
num_layers = 2
dropout_prob = 0.2 # bigger number can make underfitting.
model_lstm_10 = MyLSTMNet(num_features, hidden_layer_size, num_layers, output_size, dropout_prob)

print(model_lstm_10)
print("============================================================")

n_epochs = 201
lr = 0.001
train_loss_history_10, val_loss_history_10, model_lstm_10 = train_predict_model(model_lstm_10, n_epochs, lr, X_10, y_10, lengths_10)
print( )
vis_train_loss(train_loss_history_10, val_loss_history_10)

print( )
lookback = 10
target_col = 0
pred_eval(model_lstm_10, X_10, y_10, lengths_10, train_d_10, test_d_10, lookback, target_col)

## 3.4 Multivariate LSTM Model

## 3.4.1

In [ ]:
# Functions for no restriction to a fixed lookback
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn.utils.rnn import pad_sequence

def create_sequences(sequence, lookback, forecast_horizon, target_col, pad_value=0.0):
    T, num_features = sequence.shape
    X, y, lengths = [], [], []

    if lookback > 0:
        pad_vector = np.zeros((lookback, num_features))

        for t in range(1, T - forecast_horizon + 1):
            context = sequence[:t]
            if len(context) > lookback:
                context = context[-lookback:]
            elif len(context) == 0:
                continue  # to resolve null context problem

            padded_context = pad_vector.copy()
            padded_context[-len(context):] = context

            X.append(padded_context)
            y.append(sequence[t:t + forecast_horizon, target_col])
            lengths.append(min(len(context), lookback))

        return np.array(X), np.array(y), lengths
    else:
        for t in range(1, T - forecast_horizon + 1):
            context = torch.tensor(sequence[:t], dtype=torch.float32)
            
            lengths.append(t)

            X.append(context) # No manual padding
            y.append(torch.tensor(sequence[t:t + forecast_horizon, target_col], dtype=torch.float32))

        X_padded = pad_sequence(X, batch_first=True, padding_value=pad_value)
        y_tensor = torch.stack(y)

        return X_padded.numpy(), y_tensor.numpy(), lengths


## 3.4.2

In [ ]:
lookback = 0 # mean no restriction of lookback
test_steps = 5
target_col = 0

climate_features = ['TSDM','15D_AVG_DAILY_RAIN', '15D_AVG_MAX_TEMP', '15D_AVG_MIN_TEMP',
                    '15D_AVG_RH_TMAX', '15D_AVG_RH_TMIN','15D_AVG_EVAP_SYN', '15D_AVG_RADIATION']

X_f, y_f, lengths_f, train_d_f, test_d_f = data_prep(df, climate_features, lookback, test_steps, target_col)

print("Shape of input data after sequence creation:", X_f.shape)
print("Shape of targets after sequence creation:", y_f.shape)
print("============================================================")

num_features = X_f.shape[2]
hidden_layer_size = 20 # because of multivariate - need to increase hidden layer size
output_size = test_steps
n_epochs = 201 # to reduce running time
lr = 0.001
num_layers = 2
dropout_prob = 0.2

model_lstm_f = MyLSTMNet(num_features, hidden_layer_size, num_layers, output_size, dropout_prob)
print(model_lstm_f)
print("============================================================")

train_loss_history_f, val_loss_history_f, model_lstm_f = train_predict_model(model_lstm_f, n_epochs, lr, X_f, y_f, lengths_f)
print( )
vis_train_loss(train_loss_history_f, val_loss_history_f)
print( )
pred_eval(model_lstm_f, X_f, y_f, lengths_f, train_d_f, test_d_f, lookback, target_col)
